# Assignment 5: 75 points (plus 10 points extra credit)
## Decision Trees, Random Forest, Visualization of Predictions, and Ensembles

### IMPORTANT: 
#### You MUST read everything in tnis notebook CAREFULLY, including ALL code comments.  If you do not, then you may easily make mistakes.

This week we will build models based on decision trees.  You will need to consult some of the following in order to find out the names of model parameters for training:

https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html 

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

And we again use Yellowbrick, this time for the prediction visualizations :

https://www.scikit-yb.org/en/latest/api/classifier/class_prediction_error.html 

Be sure to review the class slides if you need to. (But read the comments in this notebook first.)



In [ ]:
# Task 1: 5 points.  Set up environment

# If some of these do not import properly, you may need to install them and re-run

import keras
import sklearn
import playsound
import tensorflow
import time

import matplotlib         as mpl   
import matplotlib.pyplot  as plt
import numpy              as np   
import pandas             as pd

from keras.datasets          import cifar10  
from playsound               import playsound
from pprint                  import pprint   

from sklearn.ensemble        import RandomForestClassifier, VotingClassifier
from sklearn.linear_model    import SGDClassifier, LogisticRegression
from sklearn.metrics         import confusion_matrix, precision_recall_curve, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_predict, cross_val_score, GridSearchCV
from sklearn.pipeline        import make_pipeline
from sklearn.preprocessing   import StandardScaler
from sklearn.svm             import LinearSVC, SVC
from sklearn.tree            import DecisionTreeClassifier, export_graphviz

from yellowbrick.classifier  import ClassBalance, ClassificationReport, ClassPredictionError, ConfusionMatrix

np.random.seed(42) 

%matplotlib inline 

'Done'

In [ ]:
# Here is the code to load and prep the CIFAR-10 data

np.random.seed(42) # Make this notebook's output stable across runs

(X_train, y_train), (X_test, y_test) = cifar10.load_data() 

# Normalize the data
X_train  = X_train.astype('float32')
X_test   = X_test.astype('float32')
X_train /= 255.0  # The largest number is 255, and the smallest 0
X_test  /= 255.0  # So this division will normalize the data.

X_train_flat = X_train.reshape(X_train.shape[0], X_train.shape[1]*X_train.shape[2]*X_train.shape[3])
X_test_flat  = X_test.reshape(X_test.shape[0],   X_test.shape[1]*X_test.shape[2]*X_test.shape[3])

# We also have to use ravel to change the target values (the values we want to predict). 
y_train = np.ravel(y_train)
y_test  = np.ravel(y_test)

LABEL_NAMES = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

'Done' 


In [ ]:
# Task 2, 5 Points

# 2 points: Create a DecisionTreeClassifier model with 'random_state' of 42
#           and a maximum depth of 9, setting its value to the variable decTree
# 2 points: Call its fit method to train the model on X_train_flat, y_train
# 1 point:  Print the accuracy score of decTree with an appropriate message 

# About 1 minute on my computer

####################  insert your code below for 5 points ####################

decTree = 




##########################  insert your code above ############################

# You will probably see a score that is about 30%


In [ ]:
# Task 3: 5 Points

# Visualizing the class prediction errors with Yellowbrick
# The URL to the yellowbrick documentation is in cell 1 

# 2 points: Set decTreeViz to the value of ClassPredictionError taking as
#           arguments: the decTree defined in the previous cell, the
#           classes set to LABEL_NAMES, and the size set to make a graph 
#           that is 640 by 480 pixels. Because decTree is already trained,
#           tell the visualizer that is_fitted is True
#           If you don't do that, you would have to re-train the model
#           here, causing you to wait unnecessarily!
# 1 point:  Call the score method on decTreeViz, passing it the test data
#           in X_test_flat and y_test
# 1 point:  Print the 'score_' property of decTreeViz (it's a property,
#           not a method, so there are no parentheses and no arguments for it)
#           In your print statement include some appropriate message so we
#           know what we are looking at.
# 1 point:  Call the show method of decTreeViz to display the graph just
#           like you would do for matplotlib


####################  insert your code below for 5 points ####################

# Instantiate the classification model and visualizer
decTreeViz = 

# If you don't tell ClassPredictionError that the model is already trained,
# then you would have to execute the following line to train it again.
# decTreeViz.fit(X_train_flat, y_train)






         

##########################  insert your code above ##########################

# You should see a warning message that would disappear if you trained the model
# here as shown in the line of code that is commented out.

# the bar colors are, bottom-up, in order of LABEL_NAMES, e.g. airplane on bottom, truck on top

In [ ]:
# The decision tree model above limited the depth of the tree to only 9 levels
# If you remove the maximum depth parameter, it will use the default
# which is unlimited!  Let's see if it does better or worse than 9

dt = DecisionTreeClassifier(random_state=42) 
dt.fit(X_train_flat,  y_train)
print('\nThe test accuracy of 1 decision tree with unrestricted depth is:', dt.score(X_test_flat, y_test)) 
# Compares to 0.3041 with depth of 9

#### How do you explain that result?  
One of the characteristics of decision trees is that they can easily overfit the training data, and that is what is happening here if we don't limit the depth of the tree.  When I had you build your first model using only 9 levels, I had previously done a grid search and found that 9 was better than 8, but accuracy started to decrease with deeper trees.

So what can we do to get better results than a single decision tree?  The answer is that we introduce democracy by using an ensemble of trees and let them all vote on the predictions, as discussed in the text and my lectures.



In [ ]:
# Task 4, 5 Points

# ENSEMBLE METHODS
# Let's first build a random forest model to compare to the single decision tree.
# We use max_depth=9 so we can compare it to the single decision tree 
# that used the same depth.

# We will build an ensemble of 100 classification trees, which is the default 
# even if you do not reference n_estimators, which controls the number of trees.
# If you change that to 500 decision trees you will only get about 
# an additional 0.5% improvement.

# Our expectation is that the ensemble will perform better than one tree.
# Since we are buildng 100 trees, we'll use n_jobs=-1 to speed up the computation, 
# building them in parallel.

# 2 points: Build a RandomForestClassifier (see documentation URL in cell 1 above)
#           Use a max depth of 9, all available CPU cores (n_jobs),
#           random state of 42 and 100 estimators.
#           We will also set verbose=True which will print progress
#           information to the screen, which helps us know your computer
#           is actually doing something useful.
#           Save your model definition into variable rf
# 1 point:  Fit rf on the training data.
# 2 points: Print the accuracy score on the test data with an appropriate message.

####################  insert your code below for 5 points ####################

rf = 







###########################  insert your code above ##########################

# This should be fast, but you may see messages created by verbose=True
# even after your print statement shows your model's test accuracy.

# On my computer this gave a score of 0.4126 vs. a single tree score about 10% points lower

#### See how much better an ensemble is?
You probably see about 10 percentage points higher accuracy with an ensemble of 100 trees.  If 100 gives us that big an improvement, how about seeing what 200 trees can do for us?  

In [ ]:
rf200 = RandomForestClassifier(max_depth=9,     n_jobs=-1, verbose=True,
                               random_state=42, n_estimators=200)

rf200.fit(X_train_flat,  y_train)

print('\nThe random forest ensemble with 200 trees and depth 9 has a test accuracy of:', 
      rf200.score(X_test_flat, y_test), '\n')


#### It seems that adding more democracy has only a small benefit, at least for random forest models at depth 9 on this dataset.

In [ ]:
# Task 5: 5 points

# Let's look at the class prediction visualizer for this model.  
# Because it's only a little bit more accurate than the previous
# model, you may not expect the graph to look much different.

# If you had trouble with Task 3, then you will have trouble here.
# If so, it's time to ask your instructor for help before you submit
# this assigment.

# The same point assignment as in Task 3 above for the 4 statements: 
#  define: 2
#  score:  1
#  print:  1 
#  show:   1

####################  insert your code below for 5 points ####################

rf200Viz = 








##########################  insert your code above ##########################


#### Compare this graph to the previous one.  There is a major difference.
Compare the height of the bars in this graph to the previous graph.  What do you notice?  See how much shorter the total height of the bars are here for birds and cats?  Even though the overal model has slightly better accuracy, there has been a major shift for these two categories, and it seems both birds and cats are now much harder to recognize than before.  Remember, each of the test data categories have 1000 actual observations, but rf200Viz only thinks it sees about 300 birds, and correctly predicts only 120.  There are also very few correct cat predictions.  So the lesson here is that the overall numeric accuracy does not give a complete picture.  You need to see more detail to know where to focus your improvements.  

Where can we see more exact detail?  In a confusion matrix, of course!

In [ ]:
# Task 6: 5 points

# You have already built a confusion matrix for assignment 3
# so go back and remind yourself how to do it, OR read the
# documentation for it in Yellowbrick here:
# https://www.scikit-yb.org/en/latest/api/classifier/index.html

# 2 points: Create a yellowbrick ConfusionMatrix, on rf200, with
#           classes set to LABEL_NAMES, and a graph size of 640 by 480.
# 2 points: Print the accuracy score of that rf200ConfMatrix on the
#           test data with an appropriate message.
# 1 point:  Show rf200ConfMatrix


####################  insert your code below for 5 points ####################

rf200ConfMatrix = 







##########################  insert your code above ##########################

In [ ]:
# Task 7: 5 points

# Since the default of 100 trees works well and is also not slow, let's push it
# and see how it does on 1000 trees, again with no restriction on the depth, but it will take 
# a lot more time to run, and your fan may run hard to cool down the CPU:

# about 3 minutes on my computer

# 2 points: Create a RandomForestClassifier, saving it to rfDeep, with
#           no maximum depth, using maximum CPU cores, 1000 trees,
#           a random state of 42 and verbose set to True.
# 2 points: Fit the model to the training data
# 1 point:  Print the accuracy score of that rfDeep on the
#           test data with an appropriate message.

####################  insert your code below for 5 points ####################

rfDeep = 








##########################  insert your code above ##########################

# you should see that more trees certainly does have an added benefit at the expense of time.

#### By combining unlimited tree depth and a lot more trees, we get the most accurate model we have tried so far!
But we still have a long way to go, as we are only getting correct predictions about half the time.  Let's look at our prediction visualizations again.

Remember above when we created only one decision tree with an unlimited depth and it overfit the training data? Notice that with an ensemble of 1000 trees it does NOT overfit the data.  Our test accuracy improved instead of getting worse.  Why is that?  It's because of the ensemble.  Adding more models acts as a regularizer. 

In [ ]:
# Task 8: 15 Points   

# On my computer this takes less than 3 minutes.

# Since this was our best model to date, let's graph once more
# both the predictions graph and the confusion matrix, both
# using the tools from Yellowbrick as before.

# 1 point:  First, print the score of rfDeep again, with an appropriate message
# 3 points: Add the code to create a ClassPredictionError object on rfDeep
# 2 points: score it on the test data 
# 2 points: and show it

# 3 points: Add the code to create a ConfusionMatrix object on rfDeep
# 2 points: score it on the test data 
# 2 points: and show it

# Remember that you are graphing the rfDeep model from the previous cell

####################  insert your code below for 15 points ####################  

# add your print statement

# Build, score and show the ClassPredictionError for rfDeep
rfDeepViz =






# Build, score and show the ConfusionMatrix for rfDeep
rfDeepCM = 







###########################  insert your code above ##########################


#### Notice how much better the predictions are for both bird and cat with rfDeep

In [ ]:
# Task 9: 25 points

# Heterogeneous Ensemble

# Just as you can have an ensemble of identical model types (e.g. random
# forest is an ensemble of decision trees), you can create an ensemble of
# completely different modeling types, which we will construct here with
# SGDClassifier, LogisticRegression and RandomForestClassifier

# If need, you can consult:
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html
# https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

# And here is the reference to manage the ensemble:
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html 

# The ensemble will likely work better if each of the participating models 
# has a similar accuracy in isolation to each of the other models. I am giving
# you various hyperoparameter values below to help achieve that goal.

# Here are the point distributions: 
# 10 points total for defining the 3 classifiers (i.e. 3.33 points for each model)
# 10 points for defining the ensemble using VotingClassifier
# 10 points to create a for-loop that fits, trains, and evaluates each model,
#           including the final ensemble (i.e. 4 models in total), 
#           and prints the resulting accuracy score of each.

# EACH of the 3 classifiers should include a random state of 42 
# and n_jobs set to the maximum number of cores.
# You have already done both of these several times by now.

# There are additional parameter suggestions for each of the 3 classifiers 
# in the comments below.

####################  insert your code below for 25 points #################### 
startTime = time.perf_counter()  

### 10 points for this subsection to define the 3 classifiers: 

# In addition to the random state and n_jobs mentioned above,
# SGDClassifier should use alpha of 0.2 and a modified_huber loss function 
sgd      = 


# In addition to the random state and n_jobs mentioned above,
# LogisticRegression should use a value of 1.5 for C (to regularize)
logReg   = 


# In addition to the random state and n_jobs mentioned above,
# RandomForestClassifier should use 200 trees and a maximum depth of only 9
rndFor   = 


### 5 points for this subsection defining the VotingClassifier:

# There will be a total of 3 keyword arguments for  VotingClassifier.
# The first will tell VotingClassifier which estimators (models)
# to use for the ensemble.  The value of that keyword should be a  
# list of Python pairs.  The second element of each pair is just the
# variable defined above for one of the model definitions,
# and the first element of the same pair should be simply 
# the same variable in quotes.  For example, the very first
# Python pair in this list should be: ('sgd', sgd).
# Now you supply the other two pairs in the list.

# The next keyword argument will tell VotingClassifier
# to use all available cores so the computation will be faster,
# and the last keyword will indicate that the ensemble 
# is to do 'hard' voting, as opposed to 'soft' voting. See text 
# and class lectures for what this means. One URL near the top 
# of this cell points to the documentation for VotingClassifier
# so you can find the correct keyword argument names.

ensemble = 


### 10 points for a loop to train, score and print the scores 
#           of the 3 individual classifiers as well as the ensemble itself.

# In Assignment 1, Task 4 you used two variables in a for-loop
# where the variables were bound by 
# two lists that were zipped together.  You will do that again here.
# The first list contains references to the 4 models, 
# i.e. [ sgd,      logReg,    rndFor,   ensemble]
# The second list contains the names of those 4 models as strings,
# i.e. ['sgd',    'logReg',  'rndFor', 'Ensemble']
# The loop variable called clf is for the first list and the
# loop variable called label is for the second list.
# Go back and LOOK at the for-loop in Task 4, Assignment 1 so
# you don't waste time on this.  It should be quite easy.

# In the body of the loop you will first call the fit method of clf, 
# passing it the training data.
# Then you will call its score method passing it the test data, 
# and saving the result in the variable clf_score
# This is followed by a print statement, which I have given to you.

for clf, label in 

    
    
    
    clf_score = 
    print("Test Accuracy: %0.2f [%s]" % (clf_score, label))
    
# This takes about 270 seconds (4.5 minutes) on my computer
stopTime  = time.perf_counter() 
print(f'Elapsed time: {stopTime - startTime:0.4f} seconds')

playsound('yourcodeisdonerunning.m4a')
    
###########################  insert your code above ##########################




In [ ]:
# Task 10:  10 extra credit points

# Make a copy of the previous cell, and modify it to perform 'soft' instead of 'hard' voting.
# Run that cell, and then look at its results.
# At the bottom of that new cell put some comments that tell me, what values you see on YOUR
# computer for:

# 1. The accuracy scores for:
#    (a) the SGD model, 
#    (b) the logistic regression model, and 
#    (c) the reandom forest model.
#    You should see the same scores for these models in both the hard voting and the soft voting cells,
#    so you only need to give these scores one time for me.

# 2. The score of your hard voting ensemble.

# 3. The score of your soft voting ensemble.

# On my computer, the hard voting ensemble did slightly better than any of the 3 
# individual models, and the soft voting ensemble did slightly better than the hard.
# However, that does not mean you will get the same results, nor does it matter
# for your grade.  I will run your code on my computer, as always.  The main thing
# I'm looking for is that your code has no logic errors and does not crash.

# That's it.  If you get the hard voting cell working properly, this will be 
# an EASY 10 points of extra credit for you.